In [1]:
import pandas as pd
import torch

In [2]:
games = pd.read_csv("/Users/brentkong/Documents/curling/data/Games.csv")
stones = pd.read_csv("/Users/brentkong/Documents/curling/data/Stones.csv")
ends = pd.read_csv("/Users/brentkong/Documents/curling/data/Ends.csv")

In [3]:
games["GameUID"] = games.apply(lambda r: f"{int(r.CompetitionID)}_{int(r.SessionID)}_{int(r.GameID)}", axis=1)
stones["GameUID"] = stones.apply(lambda r: f"{int(r.CompetitionID)}_{int(r.SessionID)}_{int(r.GameID)}", axis=1)
stones["EndUID"] = stones.apply(lambda r: f"{int(r.CompetitionID)}_{int(r.SessionID)}_{int(r.GameID)}_{int(r.TeamID)}_{int(r.EndID)}", axis=1)
stones["ShotUID"] = stones.apply(lambda r: f"{int(r.CompetitionID)}_{int(r.SessionID)}_{int(r.GameID)}_{int(r.EndID)}_{int(r.ShotID)}", axis=1)
stones.loc[stones["ShotUID"] == "24250026_18_1_9_18", "TeamID"] = 37
ends["GameUID"] = ends.apply(lambda r: f"{int(r.CompetitionID)}_{int(r.SessionID)}_{int(r.GameID)}", axis=1)
ends["TeamUID"] = ends.apply(lambda r: f"{int(r.CompetitionID)}_{int(r.SessionID)}_{int(r.GameID)}_{int(r.TeamID)}", axis=1)
ends["EndUID"] = ends.apply(lambda r: f"{int(r.CompetitionID)}_{int(r.SessionID)}_{int(r.GameID)}_{int(r.TeamID)}_{int(r.EndID)}", axis=1)


In [4]:
ends.loc[ends["Result"] == 9, "Result"] = 0
ends["PowerPlay"] = ends["PowerPlay"].fillna(0)
ends["PowerPlayBool"] = ends["PowerPlay"].replace(2, 1)
ends = ends.sort_values(["GameUID", "TeamUID", "EndID"])
ends["CumulativeScore"] = ends.groupby("TeamUID")["Result"].cumsum()
ends = ends.merge(games[["GameUID","NOC1","NOC2","TeamID1","TeamID2","LSFE"]], on="GameUID", how="left").reset_index(drop = True)
stones = stones.merge(ends[["EndUID", "Result", "PowerPlay"]], on = "EndUID", how = "left").reset_index(drop = True)


In [5]:
def assign_hammer(row):
    if row.ShotID % 2 != 0:
        return 0
    else:
        return 1

stones["Has_Hammer"] = stones.apply(assign_hammer, axis = 1)

hammer_per_end = (
    stones.sort_values("ShotID")
          .groupby("EndUID")
          .tail(1)[["EndUID", "Has_Hammer"]]
)
ends = ends.merge(hammer_per_end, on="EndUID", how="left").drop_duplicates().reset_index(drop = True)

In [6]:
def get_opp_id(row):
    if row.TeamID == row.TeamID1:
        return row.TeamID2
    if row.TeamID == row.TeamID2:
        return row.TeamID1
    return None

ends["OpponentID"] = ends.apply(get_opp_id, axis=1)

In [7]:
ends["OpponentEndUID"] = ends.apply(lambda r: f"{int(r.CompetitionID)}_{int(r.SessionID)}_{int(r.GameID)}_{int(r.OpponentID)}_{int(r.EndID)}", axis=1)

ends = ends.merge(
    ends[["EndUID", "CumulativeScore"]].rename(columns={
        "EndUID": "OpponentEndUID",
        "CumulativeScore": "OpponentCumulative"
    }),
    on="OpponentEndUID",
    how="left"
)

ends["ScoreDiff"] = ends["CumulativeScore"] - ends["OpponentCumulative"]


ends = ends.sort_values(["GameUID", "TeamID", "EndID"])

ends["PrevScoreDiff"] = (
    ends.groupby(["GameUID", "TeamID"])["ScoreDiff"]
        .shift(1)
        .fillna(0)
)


In [8]:
ends = ends.merge(
    ends[["EndUID", "Result"]].rename(columns={
        "EndUID": "OpponentEndUID",
        "Result": "OpponentResult"
    }),
    on="OpponentEndUID",
    how="left"
)
ends["EndDiff"] = ends["Result"] - ends["OpponentResult"]

ends = ends.sort_values(["GameUID", "TeamID", "EndID"])

ends["PrevEndDiff"] = (
    ends.groupby(["GameUID", "TeamID"])["EndDiff"]
        .shift(1)
        .fillna(0)
)

In [10]:
ends.head(16)

,CompetitionID,SessionID,GameID,TeamID,EndID,Result,PowerPlay,GameUID,TeamUID,EndUID,...,LSFE,Has_Hammer,OpponentID,OpponentEndUID,OpponentCumulative,ScoreDiff,PrevScoreDiff,OpponentResult,EndDiff,PrevEndDiff
0,0,10,1,20,1,1,0.0,0_10_1,0_10_1_20,0_10_1_20_1,...,0,0,22,0_10_1_22_1,0,1,0.0,0,1,0.0
1,0,10,1,20,2,0,0.0,0_10_1,0_10_1_20,0_10_1_20_2,...,0,0,22,0_10_1_22_2,3,-2,1.0,3,-3,1.0
2,0,10,1,20,3,3,0.0,0_10_1,0_10_1_20,0_10_1_20_3,...,0,1,22,0_10_1_22_3,3,1,-2.0,0,3,-3.0
3,0,10,1,20,4,0,0.0,0_10_1,0_10_1_20,0_10_1_20_4,...,0,0,22,0_10_1_22_4,7,-3,1.0,4,-4,3.0
4,0,10,1,20,5,3,0.0,0_10_1,0_10_1_20,0_10_1_20_5,...,0,1,22,0_10_1_22_5,7,0,-3.0,0,3,-4.0
5,0,10,1,20,6,0,0.0,0_10_1,0_10_1_20,0_10_1_20_6,...,0,0,22,0_10_1_22_6,7,0,0.0,0,0,3.0
6,0,10,1,20,7,1,2.0,0_10_1,0_10_1_20,0_10_1_20_7,...,0,1,22,0_10_1_22_7,7,1,0.0,0,1,0.0
7,0,10,1,20,8,0,0.0,0_10_1,0_10_1_20,0_10_1_20_8,...,0,0,22,0_10_1_22_8,10,-2,1.0,3,-3,1.0
8,0,10,1,22,1,0,0.0,0_10_1,0_10_1_22,0_10_1_22_1,...,0,1,20,0_10_1_20_1,1,-1,0.0,1,-1,0.0
9,0,10,1,22,2,3,0.0,0_10_1,0_10_1_22,0_10_1_22_2,...,0,1,20,0_10_1_20_2,1,2,-1.0,0,3,-1.0


In [26]:
def compute_geometry(board_x, board_y):
    house = [(x,y) for x,y in zip(board_x,board_y) if 0 < x < 1500 and 0 < y < 3000] # no 0 or 4095
    if not house:
        return 0,0,0,0

    # Burial depth: how close a house stone is to center
    cx, cy = 750, 800
    burial = min(((x-cx)**2 + (y-cy)**2)**0.5 for x,y in house)

    # Guard cover angle: angle between closest stone and center
    gx, gy = house[0]
    angle = torch.atan2(torch.tensor(gy-cy), torch.tensor(gx-cx)).item()

    # Clustering: how tight stones are packed
    cluster = sum(((x-cx)**2 + (y-cy)**2)**0.5 < 300 for x,y in house) / len(house)

    # Side openness: difference between left and right free space
    left_open = sum(x < cx for x,_ in house)
    right_open = sum(x > cx for x,_ in house)
    openness = right_open - left_open

    return burial, angle, cluster, openness

stones = stones.copy()
stones["BurialDepth"], stones["GuardAngle"], stones["ClusterIndex"], stones["SideOpenness"] = zip(
    *stones.apply(lambda r: compute_geometry(
        [r[f"stone_{i}_x"] for i in range(1,13)],
        [r[f"stone_{i}_y"] for i in range(1,13)]
    ), axis=1)
)

In [27]:
tendency = (
    ends[ends["PowerPlay"] != 0]  # remove 0 rows
    .groupby("OpponentID")["PowerPlay"]
    .value_counts(normalize=True)
    .unstack()
    .fillna(0)
)

tendency.columns = ["Tendency_PP_Right", "Tendency_PP_Left"]  # only 2 columns now
ends = ends.merge(tendency, on="OpponentID", how="left")
ends[["Tendency_PP_Right", "Tendency_PP_Left"]] = ends[["Tendency_PP_Right", "Tendency_PP_Left"]].fillna(0)

In [28]:
geom = stones.groupby("EndUID")[["BurialDepth","GuardAngle","ClusterIndex","SideOpenness"]].mean()
ends = ends.merge(geom, on="EndUID", how="left").fillna(0)

In [29]:
bayesian_training = ends[
    ['EndUID', "Has_Hammer",
            "PowerPlayBool",
            "EndID",
            "PrevScoreDiff", "PrevEndDiff", "Result"]
    ]

bayesian_training.to_csv("/Users/brentkong/Documents/curling/data_processing/processed_data/bayesian_training.csv", index=False)
ends.to_csv("/Users/brentkong/Documents/curling/data_processing/processed_data/ends_processed.csv", index=False)
games.to_csv("/Users/brentkong/Documents/curling/data_processing/processed_data/games_processed.csv", index=False)
stones.to_csv("/Users/brentkong/Documents/curling/data_processing/processed_data/stones_processed.csv", index=False)

In [15]:
ends.loc[6]

CompetitionID                   0
SessionID                      10
GameID                          1
TeamID                         20
EndID                           7
Result                          1
PowerPlay                     2.0
GameUID                    0_10_1
TeamUID                 0_10_1_20
EndUID                0_10_1_20_7
PowerPlayBool                 1.0
CumulativeScore                 8
NOC1                          USA
NOC2                          CZE
TeamID1                        20
TeamID2                        22
LSFE                            0
Has_Hammer                      1
OpponentID                     22
OpponentEndUID        0_10_1_22_7
OpponentCumulative              7
ScoreDiff                       1
PrevScoreDiff                 0.0
OpponentResult                  0
EndDiff                         1
PrevEndDiff                   0.0
Name: 6, dtype: object

In [ ]:
pp_varient = {"PP": ends.loc[ends["PowerPlayBool"] == 1],  "NO_PP": ends.loc[ends["PowerPlayBool"] == 0]}
final = {}
for action, variant in pp_varient.items():
    out = {}
    for end in range(1, 9):
        value_counts = variant.loc[variant["EndID"] == end]["EndDiff"].value_counts().to_dict()
        for key in value_counts:
            value_counts[key] /= len(variant.loc[variant["EndID"] == end])
        out[end] = value_counts
    final[action] = out
print(final)